# Preprocessing images

In [10]:
import numpy as np
from subprocesss import check_output
from tqdm import tqdm
from skimage.transform import resize
from skimage.morphology import label
from skimage.io import imread, imshow, imread_collection, concatenate_images


size_height = 256
size_width = 256
size_channel = 3

train_image_id = check_output(["ls", "stage1_train/"]).decode("utf8").split()
test_image_id = check_output(["ls", "stage1_test/"]).decode("utf8").split()

X_train = np.zeros((len(train_image_id), size_height, size_width, size_channel), dtype=np.uint8)
X_test = np.zeros((len(test_image_id), size_height, size_width, size_channel), dtype=np.uint8)
Y_train = np.zeros((len(train_image_id), size_height, size_width, 1), dtype=np.bool)

In [11]:
for ii, image_id in tqdm(enumerate(train_image_id), total=len(train_image_id)):
    path = "stage1_train/{}/images/{}.png".format(image_id,image_id)
    image = imread(path)[:,:,:size_channel]
    image = resize(image, (size_height, size_width), mode='constant', preserve_range=True)
    X_train[ii] = image
    mask_path = "stage1_train/{}/masks/*.png".format(image_id)
    mask = np.zeros((size_height, size_width, 1), dtype=np.bool)
    masks = imread_collection(mask_path).concatenate()
    for msk in masks:
        msk = np.expand_dims(resize(msk, (size_height, size_width), mode='constant', \
              preserve_range=True), axis=-1)
        mask = np.maximum(mask, msk)
    Y_train[ii] = mask

100%|██████████| 670/670 [04:43<00:00,  2.36it/s]


In [13]:
#fig, ax = plt.subplots(nrows=1,ncols=2, figsize=(128,128))
#ax[0].imshow(X_train[0])
#ax[1].imshow(np.squeeze(Y_train[0]))

In [14]:
for ii, image_id in tqdm(enumerate(test_image_id), total=len(test_image_id)):
    path = "stage1_test/{}/images/{}.png".format(image_id,image_id)
    image = imread(path)[:,:,:size_channel]
    image = resize(image, (size_height, size_width), mode='constant', preserve_range=True)
    X_test[ii] = image

100%|██████████| 65/65 [00:02<00:00, 29.26it/s]


# Define U-net model

In [29]:
from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers import UpSampling2D, Lambda, Merge
import tensorflow as tf
import keras
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

def unet(input_shape):
    inputs = Input(input_shape)
    X_input = Lambda(lambda x: x/255.0)(inputs)
    
    # Down block 1
    conv1 = Conv2D(16, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(X_input)
    conv2 = Conv2D(16, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    maxpool1 = MaxPooling2D(pool_size=(2,2))(conv2)
    
    # Down block 2
    conv3 = Conv2D(32, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(maxpool1)
    conv4 = Conv2D(32, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    maxpool2 = MaxPooling2D(pool_size=(2,2))(conv4)
    
    # Down block 3
    conv5 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(maxpool2)
    conv6 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    maxpool3 = MaxPooling2D(pool_size=(2,2))(conv6)
    
    # Down block 4
    conv7 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(maxpool3)
    conv8 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv7)
    maxpool4 = MaxPooling2D(pool_size=(2,2))(conv8)
    
    # Vertical block
    conv9 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(maxpool4)
    conv10 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    upsampling1 = UpSampling2D(size=(2,2))(conv10)
    concate1 = concatenate([upsampling1, conv8])
    
    # Up block 1
    conv11 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(concate1)
    conv12 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv11)
    upsampling2 = UpSampling2D(size=(2,2))(conv12)
    concate2 = concatenate([upsampling2, conv6])
    
    # Up block 2
    conv13 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(concate2)
    conv14 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv13)
    upsampling3 = UpSampling2D(size=(2,2))(conv14)
    concate3 = concatenate([upsampling3, conv4])
    
    # Up block 3
    conv15 = Conv2D(32, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(concate3)
    conv16 = Conv2D(32, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv15)
    upsampling4 = UpSampling2D(size=(2,2))(conv16)
    concate4 = concatenate([upsampling4, conv2])
    
    # Up block 4
    conv17 = Conv2D(16, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(concate4)
    conv18 = Conv2D(16, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv17)
    
    # output
    outputs = Conv2D(1, (1,1), activation='sigmoid', padding='same', kernel_initializer='he_normal')(conv18)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

In [23]:
model = unet((size_height, size_width, size_channel))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 256, 256, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_34 (Conv2D)              (None, 256, 256, 64) 1792        lambda_4[0][0]                   
__________________________________________________________________________________________________
conv2d_35 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_34[0][0]                  
__________________________________________________________________________________________________
max_poolin

In [24]:
# Define metrics and loss function
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
# Compile model
from sklearn.model_selection import train_test_split

opt = keras.optimizers.Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[mean_iou])
cb = [keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, \
     verbose=0, mode='auto', epsilon=0.005, cooldown=0, min_lr=0.000001),
#      keras.callbacks.EarlyStopping('val_loss', min_delta=0.001, patience=5, mode='min'),
      keras.callbacks.ModelCheckpoint("model.hdf5", save_best_only=True)]

XX_train, XX_cv, YY_train, YY_cv = train_test_split(X_train, Y_train, test_size=0.1, random_state=0)


# Data augmentation
data_gen_args = dict(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-6,
    rotation_range=90.0,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=1.0,
    zoom_range=0.2,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None,
    preprocessing_function=None)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(XX_train, augment=True, seed=seed)
mask_datagen.fit(YY_train, augment=True, seed=seed)

image_generator = image_datagen.flow(XX_train, batch_size=64, seed=seed)

mask_generator = mask_datagen.flow(YY_train, batch_size=64, seed=seed)

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)

history = model.fit_generator(train_generator, steps_per_epoch=len(XX_train)/64, epochs=40, \
                              validation_data=(XX_cv, YY_cv), callbacks=cb)


Epoch 1/40
